In [1]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import altair as alt
from requests.utils import quote
import os
from datetime import timedelta
from mod import alt_theme

In [2]:
fmt = "{:%Y-%m-%d}"

# Can optionally use number of days to choose dates
n_days = 60
end_date = fmt.format(pd.datetime.today())
start_date = fmt.format(pd.datetime.today() - timedelta(days=n_days))

renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]
bot_names = ["stale", "codecov", "jupyterlab-dev-mode", "henchbot"]

In [3]:
# Parameters
renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples"]
start_date = "2019-11-01"
end_date = "2019-12-01"


In [4]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [5]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [6]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [7]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [8]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [9]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (123 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (68 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (49 posts)
* [Microsoft Word Integration (Intern Project)](https://discourse.jupyter.org/t/microsoft-word-integration-intern-project) (25 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (39 posts)
* [What&rsquo;s an alternative to the word &ldquo;Fellow&rdquo; or &ldquo;Fellowship&rdquo;?](https://discourse.jupyter.org/t/whats-an-alternative-to-the-word-fellow-or-fellowship) (16 posts)
* [Benefits of the classic UI and use cases for classic over JupyterLab (Was: Why is Tim not moving to lab?)](https://discourse.jupyter.org/t/benefits-of-the-classic-ui-and-use-cases-for-classic-over-jupyterlab-was-why-is-tim-not-moving-to-lab) (12 posts)
* [Governance Office Hours Meeting Minutes](https://discourse.jupyter.org/t/governance-office-hours-meeting-minutes) (44 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)

## Active topics on the Community Forum

These are topics with lots of activity in recent history.

In [10]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (123 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (68 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (52 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (49 posts)
* [Governance Office Hours Meeting Minutes](https://discourse.jupyter.org/t/governance-office-hours-meeting-minutes) (44 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (39 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (31 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [Is there a free (even ad-supported) public JupyterHub available?](https://discourse.jupyter.org/t/is-there-a-free-even-ad-supported-public-jupyterhub-available) (26 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)

In [11]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

alt.Chart(...)

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [12]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [Kernel Packaging and Distribution](https://discourse.jupyter.org/t/kernel-packaging-and-distribution) (7 posts)
* [A new Binder GUI for launching local containers](https://discourse.jupyter.org/t/a-new-binder-gui-for-launching-local-containers) (7 posts)
* [Copy / paste in jupyterlab terminal](https://discourse.jupyter.org/t/copy-paste-in-jupyterlab-terminal) (7 posts)
* [Webbrowser module](https://discourse.jupyter.org/t/webbrowser-module) (6 posts)
* [Cant download files from within juypter notebook](https://discourse.jupyter.org/t/cant-download-files-from-within-juypter-notebook) (6 posts)
* [Seeking suggestions for the best way to implement a split-view JupyterLab notebook extension](https://discourse.jupyter.org/t/seeking-suggestions-for-the-best-way-to-implement-a-split-view-jupyterlab-notebook-extension) (4 posts)
* [Dedent selected block in text editor](https://discourse.jupyter.org/t/dedent-selected-block-in-text-editor) (4 posts)
* [Notebook widget state is lost after kernel restart](https://discourse.jupyter.org/t/notebook-widget-state-is-lost-after-kernel-restart) (4 posts)
* [Logging information from hub-spawned lab servers](https://discourse.jupyter.org/t/logging-information-from-hub-spawned-lab-servers) (3 posts)
* [How to add Markdown inside a function to pretty print equation](https://discourse.jupyter.org/t/how-to-add-markdown-inside-a-function-to-pretty-print-equation) (3 posts)

In [13]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

alt.Chart(...)

## User activity in the Community Forum

**Top posters**

These people have posted lots of comments, replies, answers, etc in the community forum.

In [14]:
def plot_user_data(users, column, sort=False):
    plt_data = users.sort_values(column, ascending=False).head(50)
    x = alt.X("username", sort=plt_data['username'].tolist()) if sort is True else 'username'
    ch = alt.Chart(data=plt_data).mark_bar().encode(
        x=x,
        y=column
    )
    return ch

In [15]:
url = "https://discourse.jupyter.org/directory_items.json"
params = {"period": "quarterly", "order": "post_count"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
users = pd.DataFrame(resp.json()['directory_items'])
users['username'] = users['user'].map(lambda a: a['username'])

In [16]:
plot_user_data(users.head(50), 'post_count')

alt.Chart(...)

**Forum users, sorted by likes given**

These are Community Forum members that "liked" other people's posts. We appreciate
anybody taking the time to tell someone else they like what they're shared!

In [17]:
plot_user_data(users.head(50), 'likes_given')

alt.Chart(...)

**Forum users, sorted by likes received**

These are folks that posted things other people in the Community Forum liked.

In [18]:
plot_user_data(users.head(50), 'likes_received')

alt.Chart(...)

In [19]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>